In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ
import numpy as np
import galois   

In [2]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [3]:
# R1CS

x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [38]:
# R1CS => QAP
target_coeffs = GF(np.array([1,2,3,4,5]))
target_polynomial = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)
# print(U_vector_of_poly_x)

def aggregate_vector_of_polynomials_with_witness(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua = aggregate_vector_of_polynomials_with_witness(U_vector_of_poly_x, witness)
Va = aggregate_vector_of_polynomials_with_witness(V_vector_of_poly_x, witness)
Wa = aggregate_vector_of_polynomials_with_witness(W_vector_of_poly_x, witness)

print(f"Ua= {Ua}")

# We can now create h(x)t(x)
hx, remainder = divmod(((Ua*Va) - Wa), target_polynomial)
assert remainder == 0, "remainder is not 0"
Ua * Va == Wa + hx * target_polynomial 
print(f"lhs = rhs so they are the same")

tau = GF(9)
Ua(tau) * Va(tau) == Wa(tau) + hx(tau) * target_polynomial(tau)
print(f"when evaluated at random number tau, theyare the same")


Ua= 7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186575808494842
lhs = rhs so they are the same
when evaluated at random number tau, theyare the same


In [125]:
# Trusted Setup
from functools import reduce
tau = GF(9)
powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(Ua.degree + 1)]
powers_of_tau_G2 = [multiply(G2, int(tau) ** i) for i in range(Ua.degree + 1)]
t_of_tau = target_polynomial(tau)
target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i * int(t_of_tau)) for i in range(target_polynomial.degree)]
print(target_powers_of_tau_G1)
# Prover evaluate QAP at Tau
# This is not encrypted
Ua_tau = Ua(tau)
Va_tau = Va(tau)
Wa_tau = Wa(tau)
hx_tau = hx(tau)
tx_tau = target_polynomial(tau)
assert Ua_tau * Va_tau == Wa_tau + hx_tau * tx_tau, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

Ua_tau_G1 = inner_product(powers_of_tau_G1, Ua.coeffs[::-1])
Va_tau_G2 = inner_product(powers_of_tau_G2, Va.coeffs[::-1])
Wa_tau_G1 = inner_product(powers_of_tau_G1, Wa.coeffs[::-1])
ht_tau = inner_product(target_powers_of_tau_G1, hx.coeffs[::-1])
print(ht_tau)

lhs = pairing(Va_tau_G2, Ua_tau_G1)
rhs = pairing(G2, add(Wa_tau_G1, ht_tau))

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

[(5136651340476158641347135049229561641726208279354069871203792715805161747619, 16837433065819275595889503804681160417798134875894572923630286174257983651115), (3608579351209469348262127153708096808392613689835619023906463782259078105056, 3310375785466101551127454278243891483882707756475491387369621652605804389332), (11915336422199313298718358060786965387251113444580415413551829178725539816878, 7169495198210300587774633191981028215643087614161813916934543087769490869999), (9948826538126424923441269225378406087889288481114770962871891348388998446354, 595268035525692565983940729933479190255995194486049690773911652150141327966), (8796560615450943079691766927107033276403917061939071706673743086759536717063, 4246612332291580202431957453865929375784000954511860209581344892369044865472)]
(10229578500132525219804232293477008740220480719254688155636045124922735668252, 9284168382991294717218307925110778314947151538513504999199960077137521427660)
verified


In [124]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha = GF(2)
beta = GF(3)
AlphaG1 = multiply(G1, int(alpha))
BetaG2 = multiply(G2, int(beta))

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
powers_of_tau_C_G1 = []
for i in range(len(U_vector_of_poly_x)):
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    powers_of_tau_C_G1.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

Cprime = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau_C_G1, witness)), Z1)
C_new = add(Cprime, ht_tau)

A_new = add(AlphaG1, Ua_tau_G1)
B_new = add(BetaG2, Va_tau_G2)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(G2,C_new)
rhs = rhs_1 * rhs_2

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")


verified


In [145]:
# separate public and private inputs
# for i in range(len(powers_of_tau_C_G1)):
#     print(powers_of_tau_C_G1[i])
# print("----")
# public input
public_witness_length = 2
public_input = []
for i in range(public_witness_length):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    public_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

print("----")
# we now have 2 points representing Bui, Avi, Wvi. Let's squash the witness in and add to 1 point
public_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(public_input, witness[:public_witness_length])), Z1)

private_input = []
for i in range(public_witness_length,len(witness)):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    private_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

# private_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(private_input, witness[public_witness_length:])), Z1)

# def log_iterations(powers, coeffs):
#     for i, (point, coeff) in enumerate(zip(powers, coeffs)):
#         print(f"Iteration: {i}, coeff: {coeff}, point: {point}")
#         yield multiply(point, int(coeff))

# private_input_w_witness3 = reduce(add, log_iterations(public_input, witness[:public_witness_length]), Z1)
# private_input_w_witness2 = reduce(add, log_iterations(private_input, witness[public_witness_length:]), Z1)

# print(private_input_w_witness)
# print(private_input_w_witness2)
# print(private_input_w_witness3)
# Cprime = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau_C_G1, witness)), Z1)
# C_new = add(Cprime, ht_tau)

(2007103093081813766129105587839348153647609275923832116616944241119959949926, 2096930367711349711597323783877123072514796801464756395332460543597665076119)
(1498339796778307471365624041433540006436600402203263068984044382637138627232, 16602075090262163391842937953477689188383027283447549643521873518378054630844)
(3105394042392028091006678147035271410126137499506258560597207370160586153422, 4975659184191176520960875954395952154279507714769779022572903275474405399622)
(5469507903858466987981420700078173472202739331512182257164535989464840481173, 10778601429035076164368616131778487282564774552652255238638437197189124686427)
(2141779991695175893126705397706683433383796298116773934577406882717181126114, 11464512254018357452934957088916697358343897658215408207230123605121399014315)
(6300917389277582963522139221161875580841814245872479853845713494168407204402, 9113513414603065448923103630098548087463328771449545261681541408332404797817)
(142721230546544577099366040421227677117463684953792485

In [130]:
# Introducing Gamma, Delta
gamma = GF(4)
delta = GF(5)
gamma_inv = np.reciprocal(gamma)
delta_inv = np.reciprocal(delta)

GammaG2 = multiply(G2, int(gamma))
DeltaG2 = multiply(G2, int(delta))

# powers of tau for public input from i = 0 to i = 1 for the 0, 1 pos of [1,out,x,y,...] 
public_witness_length = 2


# public
public_powers_of_tau_G1 = [multiply(point, int(gamma_inv)) for point in powers_of_tau_C_G1[:public_witness_length]]
gamma_pairing = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(public_powers_of_tau_G1, witness[0:public_witness_length])), Z1)


# private
private_powers_of_tau_G1 = [multiply(point, int(delta_inv)) for point in powers_of_tau_C_G1[public_witness_length+1:len(witness)]]
Cprime_delta = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(private_powers_of_tau_G1, witness[public_witness_length:len(witness)])), Z1)

#ht_tau is from i = 0 -> m. Needs to change to l -> m
delta_ht_tau = multiply(ht_tau, int(delta_inv))
C_prime_new = add(Cprime_delta, ht_tau)
C_new = add(Cprime, ht_tau)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(GammaG2, gamma_pairing)
rhs_3 = pairing(DeltaG2, C_prime_new)
rhs = rhs_1 * rhs_2 * rhs_3

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

not verified
